In [ ]:
from bs4 import BeautifulSoup
import requests
import json
import csv
from urllib.request import urlopen as uReq
import pandas as pd
import numpy as np
import re
import urllib.request
import unicodedata
import urllib.parse




1. Scrape persona name, arcana, level
2. Get ability (inherit, weak, resist, null, reflect)
3. ...





In [ ]:
# Define link
url = "https://megamitensei.fandom.com/wiki/List_of_Persona_5_Personas"
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

In [ ]:
# Find all arcana
arc = soup.find_all("span", class_="mw-headline")
print(len(arc))

24


In [ ]:
print(arc[:5])

[<span class="mw-headline" id="Fool"><a href="/wiki/Fool_Arcana" title="Fool Arcana">Fool</a></span>, <span class="mw-headline" id="Magician"><a href="/wiki/Magician_Arcana" title="Magician Arcana">Magician</a></span>, <span class="mw-headline" id="Priestess"><a href="/wiki/Priestess_Arcana" title="Priestess Arcana">Priestess</a></span>, <span class="mw-headline" id="Empress"><a href="/wiki/Empress_Arcana" title="Empress Arcana">Empress</a></span>, <span class="mw-headline" id="Emperor"><a href="/wiki/Emperor_Arcana" title="Emperor Arcana">Emperor</a></span>]


In [ ]:
persona_dict = []

for arcana_header in arc:
    arcana_name = arcana_header.get_text(strip=True)

    table = arcana_header.find_next("table", class_="table p5")

    if table:
        personas = []

        # Extract persona names from table rows
        for row in table.find_all("tr")[1:]:
            cells = row.find_all(["th","td"])

            for i in range(0, len(cells)-1,2):
              level = cells[i].get_text(strip=True)
              persona_tag = cells[i+1].find("a")

              if persona_tag:
                persona_name = persona_tag.get_text(strip=True)
                persona_dict.append((arcana_name, level, persona_name))


In [ ]:
persona_df = pd.DataFrame(persona_dict, columns=["Arcana", "Level", "Persona"])
persona_df

,Arcana,Level,Persona
0,Fool,1*,Arsène
1,Fool,8,Obariyon
2,Fool,16,High Pixie
3,Fool,20↓,Izanagi
4,Fool,23↓,Izanagi Picaro
...,...,...,...
221,Judgement,81↓,Messiah
222,Judgement,82,Shiva
223,Judgement,87,Michael
224,Judgement,90↓,Messiah Picaro


In [ ]:
persona_df.shape

(226, 3)

In [ ]:
# Retrieve persona's name only from persona_df
persona_namelist = []

arcana = arcana_header('table', {'class':'table p5'})
arcana_l = len(arcana)

for arc in range(arcana_l):
    pname = arcana[arc].findAll('a')
    pname_l = len(pname)
    for pn in range(pname_l):
        name = pname[pn].text
        if pname[pn].previous_element.previous_element.previous_element != '**':
            persona_namelist.append(name)

In [ ]:
persona_namelist = persona_df['Persona'].tolist()
persona_namelist[0]

'Arsène'

Cleaning

Get ability

In [ ]:
persona_namelist[0] = "Arsene"
persona_namelist[75] = "Kushinada-Hime"

In [ ]:
links = []
for p in persona_namelist:
    links.append("https://megamitensei.fandom.com/wiki/" + p.replace(' ', '_'))



*   Picaro personas cannot be processed because there are no main page of them
*   Some links cannot be accessed: OTW



In [ ]:
counter = 0
all_data = []
unaccessible_links = []

for link in links:
    page_url = link
    safe_url = urllib.parse.quote(page_url, safe=':/')

    try:
        with uReq(link) as uClient:
            page_content = uClient.read().decode("utf-8")

        page_soup = BeautifulSoup(page_content, "html.parser")

        # Extract the Persona name
        persona_span = page_soup.find("span", {"class": "mw-page-title-main"})
        persona_name = persona_span.text.strip() if persona_span else "Unknown"

        target_span = page_soup.find("span", {"id": "Persona_5_.2F_Royal"})

        if target_span:
            affinity_table = target_span.find_next("table", {"class": "customtable"})

            if affinity_table:
                headers = [th.get_text(strip=True) for th in affinity_table.find_all("th")]

                data = []
                for tr in affinity_table.find_all("tr")[1:]:
                    cells = [td.text.strip() for td in tr.find_all("td", recursive=False)]

                    while len(cells) < len(headers):
                        cells.append("-")  # Fill missing columns
                    while len(cells) > len(headers):
                        cells.pop()

                    # Add Persona name to the row
                    cells.insert(0, persona_name)  # Insert at the first column
                    data.append(cells)

                headers.insert(0, "Persona")

                df = pd.DataFrame(data, columns=headers).fillna("")

                df_fltr = df[~df["Arcana"].isin(["Strength", "Magic", "Endurance", "Agility", "Luck"])]
                df_fltr.reset_index(drop=True, inplace=True)

                all_data.append(df_fltr)

                counter += 1
                print(f"Processed {counter}/{len(links)}: {link}")

            else:
                print(f"No Table found for {link}")
                unaccessible_links.append(link)
        else:
            print(f"No Persona 5 / Royal section for {link}")
            unaccessible_links.append(link)

    except Exception as e:
        print(f"Error processing {link}: {e}")
        unaccessible_links.append(link)

# Combine all data into a DataFrame
if all_data:
    final_df = pd.concat(all_data, ignore_index=True)
    print("Dataframe completed")
else:
    print("¯\_(ツ)_/¯")

No Table found for https://megamitensei.fandom.com/wiki/Arsene
Processed 1/226: https://megamitensei.fandom.com/wiki/Obariyon
No Persona 5 / Royal section for https://megamitensei.fandom.com/wiki/High_Pixie
Processed 2/226: https://megamitensei.fandom.com/wiki/Izanagi
Processed 3/226: https://megamitensei.fandom.com/wiki/Izanagi_Picaro
Processed 4/226: https://megamitensei.fandom.com/wiki/Orpheus
Error processing https://megamitensei.fandom.com/wiki/Orpheus_Picaro: HTTP Error 404: Not Found
Processed 5/226: https://megamitensei.fandom.com/wiki/Decarabia
Processed 6/226: https://megamitensei.fandom.com/wiki/Legion
Processed 7/226: https://megamitensei.fandom.com/wiki/Ose
Processed 8/226: https://megamitensei.fandom.com/wiki/Bugs
No Persona 5 / Royal section for https://megamitensei.fandom.com/wiki/Crystal_Skull
Processed 9/226: https://megamitensei.fandom.com/wiki/Dionysus
Processed 10/226: https://megamitensei.fandom.com/wiki/Black_Frost
Processed 11/226: https://megamitensei.fandom.co

In [ ]:
for failed_link in unaccessible_links:
        print(failed_link)

https://megamitensei.fandom.com/wiki/Arsene
https://megamitensei.fandom.com/wiki/High_Pixie
https://megamitensei.fandom.com/wiki/Orpheus_Picaro
https://megamitensei.fandom.com/wiki/Crystal_Skull
https://megamitensei.fandom.com/wiki/Koh-i-Noor
https://megamitensei.fandom.com/wiki/Milady
https://megamitensei.fandom.com/wiki/Queen's_Necklace
https://megamitensei.fandom.com/wiki/Mother_Harlot
https://megamitensei.fandom.com/wiki/Regent
https://megamitensei.fandom.com/wiki/Phoenix
https://megamitensei.fandom.com/wiki/Bishamonten
https://megamitensei.fandom.com/wiki/Kohryu
https://megamitensei.fandom.com/wiki/Loki
https://megamitensei.fandom.com/wiki/Stone_of_Scone
https://megamitensei.fandom.com/wiki/Ariadne_Picaro
https://megamitensei.fandom.com/wiki/Asterius_Picaro
https://megamitensei.fandom.com/wiki/Orlov
https://megamitensei.fandom.com/wiki/Siegfried
https://megamitensei.fandom.com/wiki/Emperor's_Amulet
https://megamitensei.fandom.com/wiki/Hecatoncheires
https://megamitensei.fandom.com

In [ ]:
final_df

,Persona,Arcana,Level,HP,SP,Type
0,Obariyon,Fool,8,206,78,Irritable
1,Izanagi,Fool,20,NaN,NaN,NaN
2,Izanagi,Fool,20,NaN,NaN,NaN
3,Orpheus,Fool,26,NaN,NaN,NaN
4,Decarabia,Fool,32,320,222,Gloomy
...,...,...,...,...,...,...
184,Yamata-no-Orochi,Judgement,64,"1,720",519,Gloomy
185,Abaddon,Judgement,74,"3,000",387,Irritable
186,Shiva,Judgement,82,NaN,NaN,NaN
187,Michael,Justice,75,"5,600","3,720",NaN


In [ ]:
final_df.sort_values(by=['Persona'])

,Persona,Arcana,Level,HP,SP,Type
185,Abaddon,Judgement,74,"3,000",387,Irritable
70,Agathion,Chariot,3,150,54,Timid
125,Alice,Death,79,NaN,NaN,NaN
62,Ame-no-Uzume,Lovers,29,NaN,NaN,NaN
158,Ananta,Star,43,NaN,NaN,NaN
...,...,...,...,...,...,...
184,Yamata-no-Orochi,Judgement,64,"1,720",519,Gloomy
179,Yatagarasu,Sun,57,NaN,NaN,NaN
152,Yoshitsune,Tower,79,NaN,NaN,NaN
176,Yurlungur,Sun,42,NaN,NaN,NaN


In [ ]:
page_url = 'https://megamitensei.fandom.com/wiki/List_of_Persona_5_Skills'
uClient = uReq(page_url)
page_soup = BeautifulSoup(uClient.read(), "html.parser")
uClient.close()

In [ ]:
# Extract single target skills
def extract_skills_from_table(table, category, cost_index):
    skill_data = []
    if table:
        rows = table.find_all("tr")[1:]
        for row in rows:
            cols = row.find_all("td")
            if len(cols) > cost_index:
                skill = cols[0].text.strip()
                effect = cols[1].text.strip()
                cost = cols[cost_index].text.strip()
                skill_data.append([skill, effect, cost, category])
    return skill_data

all_skills_list = []

# Phys Skills
physical_skills_section = page_soup.find("span", {"id": "Physical_Skills"})
if physical_skills_section:
    physical_skills_table = physical_skills_section.find_next("table", {"class": "table p5"})
    all_skills_list.extend(extract_skills_from_table(physical_skills_table, "Physical", cost_index=5))

# Gun Skills
gun_skills_section = page_soup.find("span", {"id": "Gun"})
if physical_skills_section:
    physical_skills_table = physical_skills_section.find_next("table", {"class": "table p5"})
    all_skills_list.extend(extract_skills_from_table(physical_skills_table, "Gun", cost_index=5))

# Magic Skills
magic_categories = ["Fire", "Ice", "Electric", "Wind", "Psychokinesis", "Nuclear", "Bless", "Curse", "Almighty"]
for magic_type in magic_categories:
    magic_skills_section = page_soup.find("span", {"id": magic_type})
    if magic_skills_section:
        magic_skills_table = magic_skills_section.find_next("table", {"class": "table p5"})
        all_skills_list.extend(extract_skills_from_table(magic_skills_table, magic_type, cost_index=4))

# # Ailment Skills -- OTW
# ailment_categories = ["Physiological", "Ice", "Electric", "Wind", "Psychokinesis", "Nuclear", "Bless", "Curse", "Almighty"]
# for ailment_type in ailment_cat:
#     magic_skills_section = page_soup.find("span", {"id": magic_type})
#     if magic_skills_section:
#         magic_skills_table = magic_skills_section.find_next("table", {"class": "table p5"})
#         all_skills_list.extend(extract_skills_from_table(magic_skills_table, magic_type, cost_index=4))

df_skills = pd.DataFrame(all_skills_list, columns=["Skill", "Effect", "Cost", "Category"])


In [ ]:
df_skills

,Skill,Effect,Cost,Category
0,Lunge,Light Physical damage to 1 foe.,5% HP,Physical
1,Cleave,Light Physical damage to 1 foe.,6% HP,Physical
2,Giant Slice,Medium Physical damage to 1 foe.,9% HP,Physical
3,Assault Dive,Heavy Physical damage to 1 foe.,13% HP,Physical
4,Megaton Raid,Severe Physical damage to 1 foe.,16% HP,Physical
...,...,...,...,...
136,Stagnant Air,Increase susceptibility to all ailments of all...,5 SP,Almighty
137,Ghastly Wail,Instantly kill all foes under Fear.,28 SP,Almighty
138,Black Viper,Severe Almighty damage to 1 foe.,48 SP,Almighty
139,Morning Star,Severe Almighty damage to all foes. Unique to ...,52 SP,Almighty


In [ ]:
# # Find a specific span with id="Persona_5_/_Royal"
# # Get customtable table (Arcana, Level, HP, SP, Type)
# #

# # Define link
# url = "https://megamitensei.fandom.com/wiki/Rangda"
# headers = {"User-Agent": "Mozilla/5.0"}
# response = requests.get(url, headers=headers)
# soup = BeautifulSoup(response.text, "html.parser")

# target_span = soup.find("span", {"id": "Persona_5_/_Royal"})

# if target_span:
#     affinity_table = target_span.find_next("table", {"class": "customtable"})
#     skill_set = target_span.find_next("table", {"class": "customtable"})[2].find_all("tr")[2:]

#     if affinity_table:
#         headers_stat = [th.get_text(strip=True) for th in affinity_table.find_all("th")]
#         data = []
#         for tr in affinity_table.find_all("tr")[1:]:
#             cells = [td.text.strip() for td in tr.find_all("td", recursive=False)]
#             data.append(cells)

#         df = pd.DataFrame(data, columns=headers_stat)

#         # df_fltr = df[~df["Arcana"].isin(["Strength", "Magic", "Endurance", "Agility", "Luck"])]
#         # df_fltr.reset_index(drop=True, inplace=True)  # Reset index after deletion
#         # print(df_fltr)
#         print(df)

#     else:
#         print("No Table")
# else:
#     print("¯\_(ツ)_/¯")

,Skill,Effect,Cost
0,Lunge,Light Physical damage to 1 foe.,5% HP
1,Bicorn (talk),Cleave,20%
2,6% HP,Berith (both),90%
3,30%,9% HP,120
4,90%,20%,Severe Physical damage to 1 foe.
...,...,...,...
222,Inflict Brainwash (high odds) to 1 foe.,NaN,0 Passive
223,NaN,5 SP,0 Passive
224,Lilim (talk),Brain Jack,0 Passive
225,Inflict Brainwash (medium odds) to all foes.,NaN,0 Passive
